In [1]:
import os

In [2]:
%pwd

'h:\\Projects\\mlops\\first\\pizza-steak-end-to-end\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'h:\\Projects\\mlops\\first\\pizza-steak-end-to-end'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    update_base_model_path: Path
    training_data: Path
    params_epoch: int
    params_batch_size: int
    params_image_size: list
    params_validation_split: float

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_log_dir: Path
    checkpoint_dir: Path

In [6]:
from PizzaSteakClassifier.constants import *
from PizzaSteakClassifier.utils.common import read_yaml_file, create_directories
import tensorflow as tf

[2024-01-13 15:04:19,552 - WARNING - module_wrapper - From h:\Projects\mlops\first\pizza-steak-end-to-end\.venv\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml_file(config_file_path)
        self.params = read_yaml_file(params_file_path)

        create_directories([self.config.artifacts_root])


    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        config = self.config

        create_directories([
            Path(config.prepare_callbacks.tensorboard_log_dir),
            Path(config.prepare_callbacks.checkpoint_dir)
        ])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir = Path(config.artifacts_root),
            tensorboard_log_dir = Path(config.prepare_callbacks.tensorboard_log_dir),
            checkpoint_dir = Path(config.prepare_callbacks.checkpoint_dir)
        )

        return prepare_callback_config
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.model_training
        params = self.params
        base_model_preparation = self.config.base_model_preparation
        training_dir = os.path.join(self.config.data_ingestion.unzip_dir)

        create_directories([training.root_dir])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            update_base_model_path=Path(base_model_preparation.updated_base_model_path),
            training_data=Path(training_dir),
            params_epoch=params.epochs,
            params_batch_size=params.batch_size,
            params_image_size=params.image_size,
            params_validation_split=params.validation_split
        )

        return training_config

In [8]:
import time


In [9]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config
    
    @property
    def create_tb_callback(self):
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_log_dir,
            f"tb_logs_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def create_ckpt_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_dir), #str is required for ModelCheckpoint
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
                self.create_tb_callback,
                self.create_ckpt_callback
        ]

In [10]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.update_base_model_path
        )
    
    def data_preparation(self):
        config = self.config

        train_dataset = tf.keras.utils.image_dataset_from_directory(
            str(config.training_data),
            validation_split=config.params_validation_split,
            subset="training",
            seed=123,
            image_size=config.params_image_size[:-1],
            batch_size=config.params_batch_size
        )

        validation_dataset = tf.keras.utils.image_dataset_from_directory(
            str(config.training_data),
            validation_split=config.params_validation_split,
            subset="validation",
            seed=123,
            image_size=config.params_image_size[:-1],
            batch_size=config.params_batch_size
        )

        val_batches = tf.data.experimental.cardinality(validation_dataset)
        test_dataset = validation_dataset.take(val_batches // 5)
        validation_dataset = validation_dataset.skip(val_batches // 5)

        AUTOTUNE = tf.data.AUTOTUNE

        self.train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
        self.validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
        self.test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

    @staticmethod
    def save_model(model: tf.keras.Model, model_path: Path):
        model.save(model_path)
    
    def train(self, callback_list: list):
        self.model.fit(
            self.train_dataset,
            epochs=self.config.params_epoch,
            validation_data=self.validation_dataset,
            callbacks = callback_list
        )

        self.save_model(
            model = self.model,
            model_path = self.config.trained_model_path
        )

In [11]:
print(tf.__version__)

2.15.0


In [12]:
try:
    config=ConfigurationManager()
    prepare_callback_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callback_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.data_preparation()
    training.train(callback_list=callback_list)
    
except Exception as e:
    raise e


[2024-01-13 15:04:30,527 - INFO - common - YAML file loaded successfully: config\config.yaml]
[2024-01-13 15:04:30,530 - INFO - common - YAML file loaded successfully: params.yaml]
[2024-01-13 15:04:30,532 - INFO - common - Directory created: artifacts]
[2024-01-13 15:04:30,533 - INFO - common - Directory created: artifacts\prepare_callbacks\tensorboard_logs]
[2024-01-13 15:04:30,535 - INFO - common - Directory created: artifacts\prepare_callbacks\checkpoint_dir]
[2024-01-13 15:04:30,538 - INFO - common - Directory created: artifacts/model_training]
[2024-01-13 15:04:30,804 - WARNING - module_wrapper - From h:\Projects\mlops\first\pizza-steak-end-to-end\.venv\Lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.
]
[2024-01-13 15:04:31,247 - WARNING - module_wrapper - From h:\Projects\mlops\first\pizza-steak-end-to-end\.venv\Lib\site-packages\keras\src\layers\normal

h:\Projects\mlops\first\pizza-steak-end-to-end\.venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
